In [5]:
# import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, log_loss
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost.sklearn import XGBRegressor
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from sklearn.multioutput import MultiOutputClassifier
import os
import warnings
warnings.filterwarnings('ignore')

XGBoostLibraryNotFound: Cannot find XGBoost Library in the candidate path, did you install compilers and run build.sh in root path?
List of candidates:
C:\Users\sharm\AppData\Roaming\Python\Python37\site-packages\xgboost\xgboost.dll
C:\Users\sharm\AppData\Roaming\Python\Python37\site-packages\xgboost\../../lib/xgboost.dll
C:\Users\sharm\AppData\Roaming\Python\Python37\site-packages\xgboost\./lib/xgboost.dll
D:\Anaconda\xgboost\xgboost.dll
C:\Users\sharm\AppData\Roaming\Python\Python37\site-packages\xgboost\../../windows/x64/Release/xgboost.dll
C:\Users\sharm\AppData\Roaming\Python\Python37\site-packages\xgboost\./windows/x64/Release/xgboost.dll

In [ ]:
# reading data files and storing them in a dataframe
df_train_features = pd.read_csv('D:/Fundamental of AI/lish-moa/train_features.csv')
df_test_features = pd.read_csv('D:/Fundamental of AI/lish-moa/test_features.csv')
df_train_target_nonscored = pd.read_csv('D:/Fundamental of AI/lish-moa/train_targets_nonscored.csv')
df_train_target_scored = pd.read_csv('D:/Fundamental of AI/lish-moa/train_targets_scored.csv')

In [ ]:
# drop id col
X = df_train_features.iloc[:,1:].to_numpy()
X_test = df_test_features.iloc[:,1:].to_numpy()
y = df_train_target_scored.iloc[:,1:].to_numpy() 

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('encode', CountEncoder(cols=[0, 2])),
                ('classify', classifier)
               ])

In [ ]:
params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)

In [ ]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test.shape[0], y.shape[1]))
oof_losses = []
kf = KFold(n_splits=NFOLDS)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    # drop where cp_type==ctl_vehicle (baseline)
    ctl_mask = X_train[:,0]=='ctl_vehicle'
    X_train = X_train[~ctl_mask,:]
    y_train = y_train[~ctl_mask]
    
    clf.fit(X_train, y_train)
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
        
    print(oof_losses)
    print('Mean OOF loss across folds', np.mean(oof_losses))
    print('STD OOF loss across folds', np.std(oof_losses))

In [ ]:
# set control train preds to 0
control_mask = train['cp_type']=='ctl_vehicle'
oof_preds[control_mask] = 0

print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(oof_preds)))

In [ ]:
# set control test preds to 0
control_mask = test['cp_type']=='ctl_vehicle'

test_preds[control_mask] = 0
# create the submission file
sub.iloc[:,1:] = test_preds
sub.to_csv('submission.csv', index=False)